In [ ]:
import pandas as pd
import sys, os
import subprocess
from getpass import getpass
from langchain_huggingface import HuggingFacePipeline
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch
from huggingface_hub import snapshot_download
from huggingface_hub.utils import enable_progress_bars
project_root = os.path.abspath(os.path.join(os.getcwd(), "../"))
sys.path.append(project_root)
from src.complaintVectorStore import vector_db 
from src.complaintRagChain import create_rag_chain
from src.complaintRagChain import ComplaintRAGChain


In [ ]:
# 1. Fix the path error by using a safer pip call
print("📦 Checking dependencies...")
subprocess.check_call([sys.executable, "-m", "pip", "install", "huggingface_hub[hf_xet]"])
# 2. Force the progress bar to show
enable_progress_bars()
print("📥 Starting Phi-3-mini download (7.6GB)...")
try:
    # We set token=False to avoid the login error
    model_path = snapshot_download(
        repo_id="microsoft/Phi-3-mini-4k-instruct",
        repo_type="model",
        token=False,           # <--- This fixes your "Token is required" error
        resume_download=True,
        max_workers=4
    )
    print(f"\n✅ Success! Model files located at: {model_path}")
except Exception as e:
    print(f"\n❌ Still hitting a snag: {e}")
    print("If you see a 401 Unauthorized, you may need to run 'huggingface-cli login' in your terminal once.")

In [ ]:
print("=== Loading local LLM ===")
model_id = "microsoft/Phi-3-mini-4k-instruct"

tokenizer = AutoTokenizer.from_pretrained(model_id)

# Try to load on GPU first, fall back to CPU if memory is tight
try:
    print("Trying to load on GPU...")
    model = AutoModelForCausalLM.from_pretrained(
        model_id,
        torch_dtype=torch.float16,
        device_map="auto",
        trust_remote_code=True,
        attn_implementation="eager"
    )
except ValueError:
    print("⚠️ GPU/RAM full! Forcing CPU mode for stability...")
    model = AutoModelForCausalLM.from_pretrained(
        model_id,
        torch_dtype=torch.float32,
        device_map="cpu",
        low_cpu_mem_usage=True,
        trust_remote_code=False,
        attn_implementation="eager"
    )

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=100,
    temperature=0.1,
    do_sample=True,
    eos_token_id=tokenizer.eos_token_id,
)

llm = HuggingFacePipeline(pipeline=pipe)
print(f"✅ Loaded on: {model.device}")

In [ ]:
analyst = ComplaintRAGChain(llm=llm, vector_db=vector_db)
print("Analyst chain initialized with local Phi-3-mini")

In [ ]:
query = "What are the primary complaints regarding Savings Accounts?"

response = analyst.query(query)

print("REPORT:")
print(response['result'])
print("\nSOURCES:")
for doc in response['source_documents']:
    cid = doc.metadata.get('complaint_id', 'N/A')
    prod = doc.metadata.get('product_category', 'N/A')
    print(f"- ID: {cid} | Product: {prod}")